In [1]:

import torch 
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader,TensorDataset
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from torch.autograd import Variable
import torch.nn.functional as F

In [3]:
directory = "data\linear"
file_name = "insurance.csv"
med_path = os.path.join(directory,file_name)
med_data = pd.read_csv(med_path)

In [4]:
med_data

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [5]:
gender = {'male':1,'female':2}
region = {'southwest':1,'southeast':2,'northwest':3,'northeast':4}
smoker = {'yes':1,'no':0}
med_data.sex = [gender[item] for item in med_data.sex]
med_data.region = [region[item] for item in med_data.region]
med_data.smoker = [smoker[item] for item in med_data.smoker]

In [6]:
x = med_data.iloc[:,0:6].values
y = med_data.iloc[:,6].values

x = torch.tensor(x,dtype=torch.float32)
y = torch.tensor(y,dtype=torch.float32)
y = y.view(y.shape[0],1)
#Dataset
medDataset = TensorDataset(x,y)

#Dataloader
medLoader = DataLoader(medDataset,batch_size = 10,shuffle= True)


In [7]:
for data,label in medLoader:
    print(data)
    print(label)
    break

tensor([[26.0000,  1.0000, 30.8750,  2.0000,  0.0000,  3.0000],
        [47.0000,  1.0000, 25.4100,  1.0000,  1.0000,  2.0000],
        [30.0000,  2.0000, 32.4000,  1.0000,  0.0000,  1.0000],
        [18.0000,  1.0000, 41.1400,  0.0000,  0.0000,  2.0000],
        [38.0000,  2.0000, 30.2100,  3.0000,  0.0000,  3.0000],
        [50.0000,  1.0000, 26.6000,  0.0000,  0.0000,  1.0000],
        [32.0000,  1.0000, 30.8000,  3.0000,  0.0000,  1.0000],
        [44.0000,  1.0000, 37.1000,  2.0000,  0.0000,  1.0000],
        [45.0000,  2.0000, 25.7000,  3.0000,  0.0000,  1.0000],
        [19.0000,  2.0000, 35.1500,  0.0000,  0.0000,  3.0000]])
tensor([[ 3877.3042],
        [21978.6777],
        [ 4149.7358],
        [ 1146.7966],
        [ 7537.1641],
        [ 8444.4736],
        [ 5253.5239],
        [ 7740.3369],
        [ 9101.7979],
        [ 2134.9016]])


In [8]:
#define network
class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, hiddenSize ,outputSize):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, hiddenSize)
        self.linear2 = torch.nn.Linear(hiddenSize,outputSize)
        self.linear3 = torch.nn.Linear(inputSize,outputSize)

    def forward(self, x):
        out = self.linear3(x)
        return out

In [9]:
inputDim = 6     # takes variable 'x' 
outputDim = 1
hiddenDim = 5       # takes variable 'y'
learningRate = 0.00001 
num_epochs = 1000

model = linearRegression(inputDim, hiddenDim ,outputDim)


criterion = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)

In [14]:
loss_list = []
for epochs in range(num_epochs):
    for data,label in medLoader:
        # get output from the model, given the inputs
        outputs = model(data)
        # get loss for the predicted output
        loss = criterion(outputs, label)
        
        # get gradients w.r.t to parameters
        optimizer.zero_grad()
        loss.backward()
        
        # update parameters
        optimizer.step()
    loss_list.append(loss.data)
    if epochs % 10 == 0:
         print('epoch {}, loss {}'.format(epochs, loss.item()))

epoch 0, loss 99142784.0


In [11]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)               
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [12]:
input , label = medDataset[10]

predict_single(input,label,model)

Input: tensor([25.0000,  1.0000, 26.2200,  0.0000,  0.0000,  4.0000])
Target: tensor([2721.3208])
Prediction: tensor([7668.0977])
